In [1]:
import sys
sys.path.append("/mntc/yxy/MDPP")
import argparse
from config.config import Config
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pickle as pkl
import psutil
import threading
import os
import time
from datetime import datetime
import math
early = 0
dataset = "new_pheme"
start = time.time()
cnf = Config(dataset=dataset, version="", early=early)
print(datetime.now())

2023-08-26 13:40:34.247265


In [2]:
from utils.util import read_origin, read_retweets, time_interval

datafolder = "/mntc/yxy/MDPP/data/pheme"
origin = read_origin(cnf.originfolder)
fake = read_retweets(cnf.fakefolder)
nonfake = read_retweets(cnf.nonfakefolder)


def process_samples(Retweet, origin):
    degrees = []
    for mid, retweets in Retweet:
        if mid not in origin:
            continue
        mids = [mid]
        rel = {}
        for retweet in retweets:
            r_id = retweet["mid"]
            mids.append(r_id)
            p_id = retweet["parent"]
            if p_id not in rel:
                rel[p_id] = [r_id]
            else:
                rel[p_id].append(r_id)
        max_val = 0
        for val in rel.values():
            max_val = max(len(val), max_val)
        degrees.append(max_val)

    return degrees

fake_degrees = process_samples(fake, origin)
nonfake_degrees = process_samples(nonfake, origin)
f_max= np.max(fake_degrees)
nf_max= np.max(nonfake_degrees)
print(f_max, nf_max)

38 122


In [3]:
def read_pkl(file):
    with open(file, "rb") as f:
        data = pkl.load(f)
    return data
repr_folder = f"/mntc/yxy/MDPP/output/reprs/{dataset}"
fake_file = os.path.join(repr_folder, f"f_degree_entrophy_time.pkl")
nonfake_file = os.path.join(repr_folder, f"nf_degree_entrophy_time.pkl")
f_entrophy = read_pkl(fake_file)
nf_entrophy = read_pkl(nonfake_file)

In [4]:
f_mids = [mid for mid in f_entrophy]
nf_mids = [mid for mid in nf_entrophy]
mids = f_mids + nf_mids
print(len(mids))

5741


In [5]:
# mid = "AB8TGq5nn"
# # Y = f_entrophy["zyBxAfVCi"]
# Y = f_entrophy[mid]
# X = [i for i in range(0, len(Y))]
# plt.plot(X, Y, color="red")
# plt.show()

In [6]:
from collections import Counter
import numpy as np
import random
from datetime import datetime
import pandas as pd
# u1=2
# alpha=1
# u2=0.3
# beta=1
# down = np.exp(-u1)*alpha
# up = np.exp(-u2)*beta
# print(down, up)
def draw_stages(mid, X, Y, tags, label):
    if label == 0:
        name = "fake"
        color = "red"
    else:
        name = "nonfake"
        color = "blue"
    X_tag = []
    Y_tag = []
    for items in tags:
        X_tag.append(items[0])
        Y_tag.append(items[1])
    plt.clf()

    plt.plot(X, Y, color=color)
    plt.scatter(X_tag, Y_tag, color="black", s=10)
    plt.xlabel("Time")
    plt.ylabel("Entrophy")
    plt.show()
    plt.savefig(f"{dataset}_stages_{mid}_{name}.pdf")

In [22]:
l = []
def plot_fig(mid,X,Y,label):
    # Y = Y[:96]
    # X = X[:96]
    # ups = [0.1 * i for i in range(1,100)]
    up = 0.007
    tags = []
    # print(mid, Y)
    # print("Y:", len(Y))
    for item in range(1, len(Y)-1):
        if early != 0 and (item+1)*15 > early * 60:
            break
        d_k1 = (Y[item]-Y[item-1])/15
        d_k2 = (Y[item+1]-Y[item])/15
        #d = np.abs(d_k1-d_k2)
        d = np.abs((d_k1-d_k2)/(1+d_k1*d_k2))
        #print(d_k1, d_k2)
        # if d_k1 > d_k2:
        #     if d_k1 - d_k2 <down:
        #         tags.append((item,Y[item]))
        #         # print("tag")
        # if d_k1 < d_k2:
        #     if d_k2-d_k1 > up:
        #         tags.append((item,Y[item]))
        # print(d)
        if d > up:
            tags.append((item,Y[item]))
#     if len(tags) == 0:
#         print(mid, len(Y))
    l.append(len(Y))

    #draw_stages(mid, X,Y,tags, label)

    return tags

In [8]:
# f = ["A3V2lksJm","AcdpNkaSh","DzVjG2pXl","EdHkayXRF","znjFtDdL5"]
# t = ["AxVzS7Hku","DvNXGbng7","F916W16mf","G8JWo3UMX","HdJd4d1sH"]
# tag_list = []
# for mid in f:
#     Y = f_entrophy[mid]
#     if len(Y) > 96:
#         Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     tag = plot_fig(mid, X, Y,0)
#     tag_list.append(tag)

# for mid in t:
#     Y = nf_entrophy[mid]
#     if len(Y) > 96:
#         Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     tag = plot_fig(mid, X, Y,1)
#     tag_list.append(tag)

In [9]:
# tag_list

In [10]:
# f_selected_mids = ["AARuF0nyt","CvW2gf8M3","GdBHGhPvw"]
# nf_selected_mids = ["HoFiR7HmY","yxAQ1vStp","zoV7Ze2Vr"]
# for mid in f_selected_mids:
#     Y = f_entrophy[mid]
#     if len(Y) > 96:
#         Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     plot_fig(mid, X, Y,0)

# for mid in nf_selected_mids:
#     Y = nf_entrophy[mid]
#     if len(Y) > 96:
#         Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     plot_fig(mid, X, Y,1)

In [11]:
# Tags = {}
# fYs = []
# nfYs = []
# label_f = 0
# label_nf = 0
# random.shuffle(mids)
# cnt = 0
# for mid in mids:
#     label = -1
#     if mid in f_entrophy:
#         Y = f_entrophy[mid]
#         label = 0
#     elif mid in nf_entrophy:
#         Y = nf_entrophy[mid]
#         label = 1
#     else:
#         continue
#     if len(Y) > 96:
#         Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     plot_fig(mid, X, Y,label)
#     cnt += 1
#     if cnt>=10:
#         break

In [12]:
# Tags = {}
# fYs = []
# nfYs = []
# label_f = 0
# label_nf = 0
# random.shuffle(mids)
# selected_mids = []
# for mid in mids:
#     label = -1
#     if mid in f_entrophy:
#         Y = f_entrophy[mid]
#         label = 0
#         selected_mids.append(mid)
#     elif mid in nf_entrophy:
#         Y = nf_entrophy[mid]
#         label = 1
#         selected_mids.append(mid)
#     else:
#         continue
#     if len(Y) > 96 :
#         if label_f < 5 and label == 0:
#             fYs.append(np.array(Y[:96]))
#             label_f += 1
#         if label_nf < 5 and label == 1:
#             nfYs.append(np.array(Y[:96]))
#             label_nf += 1
#     X = [i for i in range(0, len(Y))]
#     #tags = plot_fig(mid,X,Y)
#     #Tags[mid] = tags

#     if label_f >= 5 and label_nf >= 5:
#         break
#     # if len(Tags) > 5:
#     #     break
# fYs = np.array(fYs)
# nfYs = np.array(nfYs)
# # print(Ys.shape)
# fYs = np.mean(fYs,axis=0)
# nfYs = np.mean(nfYs,axis=0)
# # print(Ys.shape)
# X = [i for i in range(0, len(fYs))]
# plt.plot(X, fYs, color="red")
# plt.plot(X, nfYs, color="blue")
# plt.xlabel("Time")
# plt.ylabel("Entrophy")
# currentDateAndTime = datetime.now()
# currentTime = currentDateAndTime.strftime("%Y-%m-%d %H:%M:%S")
# plt.savefig(f"./entrophy_figs/{dataset}-entrophy-{currentTime}.pdf")
# selected_mids = ",".join(selected_mids)
# res = {}
# res["time"] = currentTime
# res["selected_mids"] = selected_mids
# df = pd.DataFrame(res, index=[0])
# res_file = f"./entrophy_figs/selected_mids.csv"
# if os.path.exists(cnf.result_file):
#     df.to_csv(res_file, mode='a', header=False, index=False)
# else:
#     df.to_csv(res_file, mode='a', header=True, index=False)

In [13]:
# from collections import Counter
# def Draw_loglog(X, Y, labels, name):
#     X = np.array(X)
#     Y = np.array(Y)

#     fontsize = 16

#     fig, ax = plt.subplots(1,1, figsize=(5, 5))
#     ax.set_xscale("log")
#     ax.set_yscale("log")
#     ax.scatter(X, Y,s=10, c='black')
#     # ax.set_xticks(fontsize=fontsize)
#     # ax.set_yticks(fontsize=fontsize)
#     ax.set_xlabel(labels[0], fontsize=fontsize)
#     ax.set_ylabel(labels[1], fontsize=fontsize)
#     fig.savefig(f"{dataset}-{name}.pdf")

# l_Tags = []
# for mid in mids:
#     label = -1
#     if mid in f_entrophy:
#         Y = f_entrophy[mid]
#         label = 0
#     elif mid in nf_entrophy:
#         Y = nf_entrophy[mid]
#         label = 1
#     else:
#         continue
#     # if len(Y) > 96:
#     #     Y = Y[:96]
#     X = [i for i in range(0, len(Y))]
#     tags = plot_fig(mid, X, Y,label)
#     l_Tags.append(len(tags))
# # print(l_Tags[:10])
# res = Counter(l_Tags)
# X = []
# Y = []
# for k, v in res.items():
#     X.append(k)
#     Y.append(v)
# Draw_loglog(X,Y,["Number of Stages","Number of Origin Tweets"],"Stages")
# # print(res)

In [14]:
# len(nf_entrophy)

In [15]:
# def monitor_memory():
#     while True:
#         memory_info = psutil.virtual_memory()
#         print(f"Used memory: {memory_info.used / (1024 ** 3):.2f} GB")
#         # Add a suitable time delay between memory checks
#         time.sleep(1)

In [16]:
print(time.time()-start)

3.4394617080688477


In [24]:
3.43+1.41

4.84

In [23]:
# memory_thread = threading.Thread(target=monitor_memory)
# memory_thread.start()
start = time.time()
Tags = {}
fYs = []
nfYs = []
label_f = 0
label_nf = 0
random.shuffle(mids)
cnt = 0
for mid in mids:
    label = -1
    if mid in f_entrophy:
        Y = f_entrophy[mid]
        label = 0
    elif mid in nf_entrophy:
        Y = nf_entrophy[mid]
        label = 1
    else:
        continue
#     if len(Y) > 96:
#         Y = Y[:96]
    X = [i for i in range(0, len(Y))]
    tags = plot_fig(mid, X, Y,label)
    Tags[mid] = tags
    '''cnt += 1
    if cnt>=10:
        break'''
tagfile = cnf.tagsfile
# print(tagfile)
with open(tagfile, "wb") as f:
    pkl.dump(Tags,f)
print(time.time()-start)
# memory_thread.join()

1.4078304767608643


In [18]:
len(Tags)

5741

In [19]:
#if early==0:
#     tagfile = os.path.join(cnf.datafolder,"tags.pkl")
# else:
#     tagfile = os.path.join(cnf.datafolder,f"tags_{str(early)}.pkl")



In [20]:
# 各样本分了多少阶段
cnt = 0
stages = []
for k,v in Tags.items():
    stages.append(len(v))
res = Counter(stages)
tmp = []
for k,v in res.items():
    tmp.append((k,v))
tmp = sorted(tmp, key=lambda x:x[0])
for item in tmp:
    print(item)

(0, 1135)
(1, 821)
(2, 696)
(3, 701)
(4, 497)
(5, 458)
(6, 344)
(7, 276)
(8, 193)
(9, 148)
(10, 99)
(11, 94)
(12, 71)
(13, 57)
(14, 39)
(15, 25)
(16, 30)
(17, 21)
(18, 12)
(19, 8)
(20, 1)
(21, 5)
(22, 2)
(23, 3)
(24, 3)
(25, 1)
(28, 1)


In [21]:
res

Counter({0: 1135,
         4: 497,
         2: 696,
         1: 821,
         5: 458,
         10: 99,
         6: 344,
         3: 701,
         7: 276,
         8: 193,
         16: 30,
         9: 148,
         15: 25,
         21: 5,
         14: 39,
         11: 94,
         12: 71,
         13: 57,
         18: 12,
         17: 21,
         20: 1,
         23: 3,
         19: 8,
         25: 1,
         24: 3,
         28: 1,
         22: 2})